In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
import tvm
from tvm import relay
from tvm.relay.dataflow_pattern import rewrite
from tvm.relay.dataflow_pattern import *
import numpy as np
from test_model import Model
from tvm.relay import transform
from tvm.relay.testing import run_opt_pass
from tvm.relay import transform, build_module
from tvm.relay import testing
import tvm.testing

/home/jd/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_network():
    img_size = 8
    out_channels = 16
    batch_size = 1
    data = relay.var("data", relay.TensorType((batch_size, 3, img_size, img_size), "float16"))
    dense_weight = relay.var(
        "dweight", relay.TensorType((batch_size, 16 * img_size * img_size), "float16")
    )
    weight = relay.var("weight")
    second_weight = relay.var("second_weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")
    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.nn.batch_flatten(simple_net)
    simple_net = relay.nn.dense(simple_net, dense_weight)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
    data_shape = (batch_size, 3, img_size, img_size)
    net, params = testing.create_workload(simple_net)
    return net, params, data_shape




In [57]:
def get_skip_net():
    img_size = 8
    out_channels = 16
    batch_size = 1
    data = relay.var("data", relay.TensorType((batch_size, 3, img_size, img_size), "float16"))
    dense_weight = relay.var(
        "dweight", relay.TensorType((batch_size, 16 * img_size * img_size), "float16")
    )
    weight = relay.var("weight")
    second_weight = relay.var("second_weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")
    x0 = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    x1 = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.op.concatenate(relay.Tuple([x0, x1]), axis=0)
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.nn.batch_flatten(simple_net)
    simple_net = relay.nn.dense(simple_net, dense_weight)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
    data_shape = (batch_size, 3, img_size, img_size)
    net, params = testing.create_workload(simple_net)
    return net, params, data_shape

print(get_skip_net()[0])

def @main(%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(2, 1), float16] {
  %0 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %2 = (%0, %1) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(1, 16, 8, 8), float16]) */;
  %3 = concatenate(%2) /* ty=Tensor[(2, 16, 8, 8), float16] */;
  %4 = nn.batch_norm(%3, %bn_gamma, %bn_beta, %bn_mean, %bn_

In [63]:
body = get_skip_net()[0]['main'].body
body = run_opt_pass(body, transform.ToANormalForm())
body = run_opt_pass(body, transform.InferType())

In [64]:
body

LetNode(Var(x_51), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6d1fb98), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_52), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c03a68), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_53), Tuple([Var(x_51), Var(x_52)]), LetNode(Var(x_54), CallNode(Op(concatenate), [Var(x_53)], relay.attrs.ConcatenateAttrs(0x6cbe958), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]), LetNode(Var(x_55), CallNode(Op(nn.batch_norm), [Var(x_54), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs

In [ ]:
LetNode(
    Var(x_51), 
    CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6d1fb98), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
    LetNode(
        Var(x_52), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c03a68), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
        LetNode(
            Var(x_53), 
            Tuple([Var(x_51), Var(x_52)]), 
            LetNode(
                Var(x_54), 
                CallNode(Op(concatenate), [Var(x_53)], relay.attrs.ConcatenateAttrs(0x6cbe958), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]), 
                LetNode(
                    Var(x_55), 
                    CallNode(Op(nn.batch_norm), [Var(x_54), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6bfd428), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 
                    LetNode(Var(x_56), TupleGetItemNode(Var(x_55), 0), LetNode(Var(x_57), CallNode(Op(nn.relu), [Var(x_56)], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_58), CallNode(Op(nn.batch_flatten), [Var(x_57)], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_59), CallNode(Op(nn.dense), [Var(x_58), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6bf3428), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)]), Var(x_59))))))))))

In [65]:
LetNode(
    Var(x_42, ty=TensorType([1, 16, 8, 8], float16)), 
    CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c7b508), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
    ), 
    LetNode(
        Var(x_43, ty=TensorType([1, 16, 8, 8], float16)), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x6c2a268), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
        ), 
        LetNode(
            Var(x_44, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])), 
            Tuple([Var(x_42, ty=TensorType([1, 16, 8, 8], float16)), Var(x_43, ty=TensorType([1, 16, 8, 8], float16))]
            ), 
            LetNode(
                Var(x_45, ty=TensorType([2, 16, 8, 8], float16)), 
                CallNode(Op(concatenate), [Var(x_44, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)]))], relay.attrs.ConcatenateAttrs(0x6c938b8), [TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([1, 16, 8, 8], float16)])]
                ), 
                LetNode(
                    Var(x_46, ty=TupleTypeNode([TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 
                    CallNode(Op(nn.batch_norm), [Var(x_45, ty=TensorType([2, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6cb11b8), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]
                    ), 
                    LetNode(Var(x_47, ty=TensorType([2, 16, 8, 8], float16)), TupleGetItemNode(Var(x_46, ty=TupleTypeNode([TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), LetNode(Var(x_48, ty=TensorType([2, 16, 8, 8], float16)), CallNode(Op(nn.relu), [Var(x_47, ty=TensorType([2, 16, 8, 8], float16))], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_49, ty=TensorType([2, 1024], float16)), CallNode(Op(nn.batch_flatten), [Var(x_48, ty=TensorType([2, 16, 8, 8], float16))], (nullptr), [TensorType([2, 16, 8, 8], float16)]), LetNode(Var(x_50, ty=TensorType([2, 1], float16)), CallNode(Op(nn.dense), [Var(x_49, ty=TensorType([2, 1024], float16)), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6b79f78), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)]), Var(x_50, ty=TensorType([2, 1], float16)))))))))))

NameError: name 'LetNode' is not defined

In [ ]:
[TupleGetItemNode(
    CallNode(Op(nn.batch_norm), 
        [CallNode(Op(concatenate), 
        [Tuple([CallNode(Op(nn.conv2d), 
        [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x6d34998), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), 
        CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x6bfeed8), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)])])], 
        relay.attrs.ConcatenateAttrs(0x6c1cbe8), 
        [TupleTypeNode([TensorType([1, 16, 8, 8], float16), 
        TensorType([1, 16, 8, 8], float16)])]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x6c05b58), [TensorType([2, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([2, 16, 8, 8], float16)])], (nullptr), [TensorType([2, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x6c6c018), [TensorType([2, 1024], float16), TensorType([1, 1024], float16)])

In [4]:
net, params, data_shape = get_network()

In [14]:
net['main'].body

CallNode(Op(nn.dense), [CallNode(Op(nn.batch_flatten), [CallNode(Op(nn.relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([1, 16, 8, 8], float16)])], (nullptr), [TensorType([1, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)])

In [ ]:
CallNode(Op(nn.dense), [CallNode(Op(nn.batch_flatten), [CallNode(Op(nn.relu), [TupleGetItemNode(CallNode(Op(nn.batch_norm), [CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(
    bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), 0)], (nullptr), [TensorType([1, 16, 8, 8], float16)])], (nullptr), [TensorType([1, 16, 8, 8], float16)]), Var(dweight, ty=TensorType([1, 1024], float16))], relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)])


In [20]:
# lets explore
body = net['main'].body
body = run_opt_pass(body, transform.ToANormalForm())
body = run_opt_pass(body, transform.InferType())
cur = body

In [25]:
print(body)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
let %x_0: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
free_var %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
let %x_1: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_0, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;

In [37]:
cur = body

In [21]:
pre = cur
cur = cur.value
print(cur)
print(type(cur))

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */
<class 'tvm.relay.expr.Call'>


In [39]:
type(cur)

tvm.relay.expr.Let

In [40]:
cur

LetNode(Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]), LetNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), CallNode(Op(nn.batch_norm), [Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), LetNode(Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), 

In [41]:
cur.var

Var(x_0, ty=TensorType([1, 16, 8, 8], float16))

In [42]:
cur.value

CallNode(Op(nn.conv2d), [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], relay.attrs.Conv2DAttrs(0x65c2968), [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)])

In [43]:
cur.body

LetNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), CallNode(Op(nn.batch_norm), [Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), Var(bn_gamma, ty=TensorType([16], float16)), Var(bn_beta, ty=TensorType([16], float16)), Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))], relay.attrs.BatchNormAttrs(0x331cbf8), [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]), LetNode(Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), LetNode(Var(x_3, ty=TensorType([1, 16, 8, 8], float16)), CallNode(Op(nn.relu), [Var(x_2, ty=TensorType([1, 16, 8, 8], float16))], (nullptr), [TensorType([1, 16, 8, 8], float16)]), LetNode(Var(x_4, ty=TensorType([1, 1024], float16)), 

In [44]:
cur.span

In [28]:
LetNode(
    Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), 
    CallNode(
        Op(nn.conv2d), 
        [Var(data, ty=TensorType([1, 3, 8, 8], float16)), Var(weight, ty=TensorType([16, 3, 3, 3], float16))], 
        relay.attrs.Conv2DAttrs(0x65c2968), 
        [TensorType([1, 3, 8, 8], float16), TensorType([16, 3, 3, 3], float16)]
    ), 
    LetNode(
        Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 
        CallNode(
            Op(nn.batch_norm), 
            [
                Var(x_0, ty=TensorType([1, 16, 8, 8], float16)), 
                Var(bn_gamma, ty=TensorType([16], float16)), 
                Var(bn_beta, ty=TensorType([16], float16)), 
                Var(bn_mean, ty=TensorType([16], float16)), Var(bn_var, ty=TensorType([16], float16))
            ], 
            relay.attrs.BatchNormAttrs(0x331cbf8), 
            [TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16), TensorType([16], float16)]
        ), 
        LetNode(
            Var(x_2, ty=TensorType([1, 16, 8, 8], float16)), 
            TupleGetItemNode(Var(x_1, ty=TupleTypeNode([TensorType([1, 16, 8, 8], float16), TensorType([16], float16), TensorType([16], float16)])), 0), 
            LetNode(
                Var(x_3, ty=TensorType([1, 16, 8, 8], float16)), 
                CallNode(
                    Op(nn.relu), 
                    [Var(x_2, ty=TensorType([1, 16, 8, 8], float16))], 
                    (nullptr), 
                    [TensorType([1, 16, 8, 8], float16)]
                ), 
                LetNode(
                    Var(x_4, ty=TensorType([1, 1024], float16)), 
                    CallNode(
                        Op(nn.batch_flatten), 
                        [Var(x_3, ty=TensorType([1, 16, 8, 8], float16))], 
                        (nullptr), 
                        [TensorType([1, 16, 8, 8], float16)]
                    ), 
                    LetNode(
                        Var(x_5, ty=TensorType([1, 1], float16)), 
                        CallNode(
                            Op(nn.dense), 
                            [Var(x_4, ty=TensorType([1, 1024], float16)), Var(dweight, ty=TensorType([1, 1024], float16))], 
                            relay.attrs.DenseAttrs(0x331d478), [TensorType([1, 1024], float16), TensorType([1, 1024], float16)]
                        ), 
                        Var(x_5, ty=TensorType([1, 1], float16))
                    )
                )
            )
        )
    )
)


NameError: name 'LetNode' is not defined

In [6]:
def graph_split(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    if (
                        split_conf
                        and split_operator_name in operator_index_map
                        and operator_index_map[split_operator_name] >= split_operator_index
                    ):
                        # Do graph splitting.
                        split_conf.pop(0)
                        snode_dep.append({"nodes": {}, "ref_nodes": {}})
                        ann = _recursion(
                            anf.body,
                            pipeline_mods,
                            split_conf,
                            constant_expr,
                        )
                        snode_dep.pop()
                        dep_vars = get_dep_var(snode_dep)
                        # When the nodes of the current subgraph are the depedency node of another
                        # subgraph, we need to set them as the output of current subgraph.
                        body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                        # when the operator of current subgraph uses previous subgraph constant
                        # as the argument of a "relay.expr.call", such constant may become a free
                        # varaible if the constant does not exist in the current subgraph.
                        # merge the previous constant with current subgraph to avoid such issue.
                        if constant_expr:
                            ann = merge_constant_expr(constant_expr, ann)
                        ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                        mod = tvm.IRModule.from_expr(ann)
                        pipeline_mods.insert(0, mod)
                        # Return the last node of the current subgraph.
                        return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, mod)
    return pipeline_mods


In [7]:
split_config = [{"op_name": "nn.relu", "op_index": 0}]
print(net['main'])
subgraphs = graph_split(net["main"], split_config, params)

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(1, 1), float16] {
  %0 = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.batch_norm(%0, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;
  %2 = %1.0 /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %3 = nn.relu(%2) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %4 = nn.batch_flatten(%3) /* ty=Tensor[(1, 1024), float16] */;
  nn.dense(%4, %dw

In [9]:
len(subgraphs)

2

In [21]:
netnet = run_opt_pass(net["main"], transform.ToANormalForm())
netnet = run_opt_pass(netnet, transform.InferType())

In [34]:
print(netnet)

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */, %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */, %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */, %dweight: Tensor[(1, 1024), float16] /* ty=Tensor[(1, 1024), float16] */) -> Tensor[(1, 1), float16] {
  let %x_12: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  let %x_13: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_12, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8),

In [32]:
print(netnet.body)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
let %x_12: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */ = nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
free_var %bn_gamma: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_beta: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_mean: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
free_var %bn_var: Tensor[(16), float16] /* ty=Tensor[(16), float16] */;
let %x_13: (Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */ = nn.batch_norm(%x_12, %bn_gamma, %bn_beta, %bn_mean, %bn_var) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) 

In [33]:
print(netnet.body.value)

free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
free_var %weight: Tensor[(16, 3, 3, 3), float16] /* ty=Tensor[(16, 3, 3, 3), float16] */;
nn.conv2d(%data, %weight, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */


In [35]:
print(type(netnet.body.value.args))
print(netnet.body.value.args[0])

<class 'tvm.ir.container.Array'>
free_var %data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */;
%data


In [16]:
subgraphs[0]['main']

fn (%data: Tensor[(1, 3, 8, 8), float16] /* ty=Tensor[(1, 3, 8, 8), float16] */) {
  %0 = nn.conv2d(%data, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float16] */, padding=[1, 1, 1, 1], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 8, 8), float16] */;
  %1 = nn.batch_norm(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float16] */, meta[relay.Constant][2] /* ty=Tensor[(16), float16] */, meta[relay.Constant][3] /* ty=Tensor[(16), float16] */, meta[relay.Constant][4] /* ty=Tensor[(16), float16] */) /* ty=(Tensor[(1, 16, 8, 8), float16], Tensor[(16), float16], Tensor[(16), float16]) */;
  %2 = %1.0 /* ty=Tensor[(1, 16, 8, 8), float16] */;
  nn.relu(%2) /* ty=Tensor[(1, 16, 8, 8), float16] */
}



In [14]:
subgraphs[1]['main']

fn (%data_n_0: Tensor[(1, 16, 8, 8), float16] /* ty=Tensor[(1, 16, 8, 8), float16] */) {
  %0 = nn.batch_flatten(%data_n_0) /* ty=Tensor[(1, 1024), float16] */;
  nn.dense(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 1024), float16] */, units=None) /* ty=Tensor[(1, 1), float16] */
}

In [2]:
model_keras = Model(3, 1, 16)

In [3]:
target = 'cuda'
dev = tvm.cuda()
img_size = 256
input_data = np.random.normal(0,1,(1,img_size,img_size,3)).astype(np.float32)
# tvm result
input_data = input_data.transpose([0, 3, 1, 2])

shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [7]:
mod['main']

fn (%input_1: Tensor[(1, 3, 256, 256), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32]) {
  %0 = nn.conv2d(%input_1, %v_param_1, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %1 = nn.bias_add(%0, %v_param_2);
  %2 = nn.batch_norm(%1, %v_param_3, %v_param_4, %v_param_5, %v_param_6, epsilon=0.001f);
  %3 = %2.0;
  %4 = nn.leaky_relu(%3, alpha=0.2f);
  %5 = nn.conv2d(%4, %v_param_7, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]);
  %6 = nn.bias_add(%5, %v_param_8);
  %7 = nn.batch_norm(%6, %v_param_9, %v_param_10, %v_param_11, %v_param_12, epsilon=0.

In [29]:
from tvm.relay.op.annotation import compiler_begin, compiler_end

def test_extern_ccompiler_single_op():
    @transform.function_pass(opt_level=0)
    class MyAnnotator:
        def transform_function(self, func, mod, dev):
            class Annotator(tvm.relay.ExprMutator):
                def visit_call(self, call):
                    new_args = []
                    for arg in call.args:
                        ann = compiler_begin(self.visit(arg), "ccompiler")
                        new_args.append(ann)
                    new_call = relay.Call(call.op, new_args)
                    return compiler_end(new_call, "ccompiler")

            return Annotator().visit(func)

    x = relay.var("x", shape=(8, 8))
    y = relay.var("y", shape=(8, 8))
    z = x + y
    z = z + relay.const(9.0)
    z = z + relay.const(9.0)
    f = relay.Function([x, y], z)
    x_data = np.random.rand(8, 8).astype("float32")
    y_data = np.random.rand(8, 8).astype("float32")
    mod = tvm.IRModule()
    mod["main"] = f
    mod = MyAnnotator()(mod)
    mod = transform.PartitionGraph()(mod)

    # print(mod)
    # print("---------------")
    # print({"x": x_data, "y": y_data}, (8, 8), x_data + y_data)
    return mod

In [30]:
mm = test_extern_ccompiler_single_op()


In [89]:
type(mm)

tvm.ir.module.IRModule

In [92]:
mod = tvm.IRModule.from_expr(mm['tvmgen_default_ccompiler_main_0'])


In [93]:
mod

#[version = "0.0.5"]
def @tvmgen_default_ccompiler_main_0(%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
}

In [94]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


ValueError: Traceback (most recent call last):
  7: TVMFuncCall
  6: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  5: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  4: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  3: tvm::relay::backend::RelayBuildModule::OptimizeImpl(tvm::IRModule)
  2: tvm::relay::backend::BindParamsInModule(tvm::IRModule, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> 

In [96]:
z = mm.get_global_vars()[0]
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: <class 'tvm.ir.expr.GlobalVar'> has no attribute params

In [88]:
for func in mm.functions:
    print(func.astext().split('\n')[-1].split('@')[-1])

tvmgen_default_ccompiler_main_4
tvmgen_default_ccompiler_main_0
tvmgen_default_ccompiler_main_2
main


In [46]:
x = relay.var("x", shape=(8, 8))
y = relay.var("y", shape=(8, 8))
z = mm['tvmgen_default_ccompiler_main_0'](x,y)
f = relay.Function([x, y], z)
mod1 = tvm.IRModule()
mod1['main'] = f

In [69]:
type(z)

tvm.relay.function.Function

In [72]:
z = mm['tvmgen_default_ccompiler_main_0']
z.attrs.global_symbol = 'main'
z = z.with_attr("global_symbol", "main")
mod = tvm.IRModule(z)
# mod['main'] = z
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)



TVMError: Traceback (most recent call last):
  2: TVMFuncCall
  1: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)>::AssignTypedLambda<tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}>(tvm::{lambda(tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void>, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void>)#2}, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::TVMRetValue)
  0: tvm::runtime::TVMMovableArgValueWithContext_::operator tvm::runtime::Ma

In [ ]:
# func0 = func0.with_attr("global_symbol", "main")


In [49]:
mod1

#[version = "0.0.5"]
def @main(%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}

In [73]:
z = mm['tvmgen_default_ccompiler_main_0']
f = relay.Function(z.params, z.body)
mod = tvm.IRModule()
mod['main'] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

AttributeError: 'GraphExecutorFactoryModule' object has no attribute 'body'

In [61]:
f

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */) {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
}

In [59]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod2, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [40]:
print(mm['tvmgen_default_ccompiler_main_0'])

fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
  add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
} /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */


In [39]:
print(f)

fn (%x: Tensor[(8, 8), float32], %y: Tensor[(8, 8), float32]) {
  %0 = fn (%ccompiler_0_i0: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, %ccompiler_0_i1: Tensor[(8, 8), float32] /* ty=Tensor[(8, 8), float32] */, Inline=1, Compiler="ccompiler", global_symbol="tvmgen_default_ccompiler_main_0", Primitive=1) -> Tensor[(8, 8), float32] {
    add(%ccompiler_0_i0, %ccompiler_0_i1) /* ty=Tensor[(8, 8), float32] */
  } /* ty=fn (Tensor[(8, 8), float32], Tensor[(8, 8), float32]) -> Tensor[(8, 8), float32] */;
  %0(%x, %y)
}


In [15]:
mod = tvm.IRModule()
mod["main"] = f
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)
    # lib = relay.build(mod2, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [16]:
lib

In [78]:
print(type(legion))


<class 'tvm.relay.function.Function'>


In [80]:
print(type(mm['main']))


<class 'tvm.relay.function.Function'>


In [43]:
# f = type(mm['tvmgen_default_ccompiler_main_0'])
# mod = tvm.IRModule()
# mod["main"] = mm['tvmgen_default_ccompiler_main_0']
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)


In [33]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


In [11]:
# legion = mm['tvmgen_default_ccompiler_main_0']
# mod2 = tvm.IRModule()
# mod2["main"] = legion
with tvm.transform.PassContext(opt_level=3):
    # lib = relay.build(mm['tvmgen_default_ccompiler_main_0'], target, params=params,mod_name='tvmgen_default_ccompiler_main_0')
    lib = relay.build(mm, target, params=params,mod_name='tvmgen_default_ccompiler_main_0')


ValueError: Traceback (most recent call last):
  8: TVMFuncCall
  7: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  6: tvm::relay::backend::RelayBuildModule::Build(tvm::IRModule, tvm::runtime::Array<tvm::Target, void> const&, tvm::Target const&, tvm::relay::Executor const&, tvm::relay::Runtime const&, tvm::WorkspaceMemoryPools const&, tvm::ConstantMemoryPools const&, tvm::runtime::String)
  5: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, tvm::runtime::String const&)
  4: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::relay::backend::GraphExecutorCodegenModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)

In [65]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mm, target, params=params)


/home/jd/workspace/tvm-v0.9.0/python/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


In [26]:
lib

In [45]:
lib['get_graph_json']

In [62]:
graph_json_raw = lib['get_graph_json']()


In [19]:
import pygraphviz as pgv
import json
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            # if args.show_size == 1:
            if 1 == 1:
                src_size = 1
                for i in json_data['attrs']['shape'][1][src[0]]:
                    src_size = src_size * i
                
                dst_size = 1
                for i in json_data['attrs']['shape'][1][node_idx]:
                    dst_size = dst_size * i
                # print(src[0], json_data['nodes'][src[0]]['name'], src_size)

                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]) + "[{}]".format(src_size), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]) + "[{}]".format(dst_size))
            else:
                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

In [63]:
show_graph(graph_json_raw, 'test')
